# Week 13 (Evaluation and Deployment )

---

### Buat model sesuai dengan topik final project dengan metode yang berbeda untuk setiap anggota kelompok. Lanjutkan Deploy dengan menggunakan Streamlit atau Flask. Tuliskan Code, tahapan pembuatan deploy dan link url hasil deploy disini. 

In [5]:
pip install kagglehub

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 1. Download Dataset From Kaggle with kagglehub
import kagglehub
import os

path = kagglehub.dataset_download("nazmul0087/ct-kidney-dataset-normal-cyst-tumor-and-stone")
print("Raw dataset folder:", path)

# --- Find folder containing classes ---
DATASET_DIR = None
for root, dirs, files in os.walk(path):
    if all(cls in dirs for cls in ["Cyst", "Normal", "Stone", "Tumor"]):
        DATASET_DIR = root
        break

if DATASET_DIR is None:
    raise Exception("❌ Folder kelas tidak ditemukan!")

print("✅ Dataset folder:", DATASET_DIR)

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

IMG_SIZE = (96, 96)      
BATCH_SIZE = 8            
EPOCHS = 3                
CLASS_NAMES = ["Cyst", "Normal", "Stone", "Tumor"]

# 3. Data Generator
datagen = ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.2,
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True
)

train_data = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

val_data = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

# 4. Build MobileNetV2
base_model = MobileNetV2(
    input_shape=(*IMG_SIZE, 3),
    include_top=False,
    weights="imagenet",
    alpha=0.35  
)

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(32, activation='relu'),   # 🔥 kecil
    layers.Dropout(0.2),
    layers.Dense(4, activation='softmax')
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS
)

os.makedirs("model", exist_ok=True)
model.save("model/kidney_model.h5")

print("🎉 Model kecil berhasil disimpan: model/kidney_model.h5")


/opt/homebrew/Cellar/jupyterlab/4.4.2/libexec/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 48%|██████████████████████████▎                            | 744M/1.52G [17:38<16:51, 838kB/s]